In [1]:
# 0.0. Importando as libs

from matplotlib.dates import DateFormatter, AutoDateLocator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

# 0.1. Construindo a path de input e importanto o dataframe

script_path = os.getcwd()
cutting_string = "Scripts"

partes = script_path.split(cutting_string)
path_insumos = partes[0] + "Insumos\\Semi-tratado (v2)"

df = pd.read_excel(path_insumos + "\\Semitratado.xlsx")

# Convertendo a coluna de datas
#df['Data da Coleta'] = pd.to_datetime(df['Data da Coleta'])

In [2]:
df.columns

Index(['Data da Coleta', 'Valor de Venda Gasolina (p/L)',
       'Valor de Venda Etanol (p/L)', 'Valor de Venda Diesel (p/L)',
       'Valor de Venda GLP (p/13kg)', 'Cotação dolar em Real (Compra)',
       'Cotação dolar em Real (Venda)', 'Cotação riyal em Real (Compra)',
       'Cotação riyal em Real (Venda)', 'Fechamento ARAMCO (em Riyais)',
       'Volume de transações ARAMCO (Em milhares)',
       'Fechamento CHEVRON (em Reais)',
       'Volume de transações CHEVRON (Em milhares)',
       'Fechamento EXXON (em Reais)',
       'Volume de transações EXXON (Em milhares)',
       'Fechamento PETR3 (em Reais)',
       'Volume de transações PETR3 (Em milhares)',
       'Fechamento PETR4 (em Reais)',
       'Volume de transações PETR4 (Em milhares)'],
      dtype='object')

In [3]:
# O intervalo de corte de outlier é uma variável que deve ser declarada a fim de agrupar os dados
# de x em x dias para detectar outliers nesse meio. A ideia é, com isso, suavizar a curva a medida
# que aumentamos o intervalo de corte de outlier, levando em conta que quanto maior a amostra, 
# mais discrepâncias serão detectadas.
intervalo_de_corte_outliers = 15

# 1.0. Construindo a função que anula outliers
def anula_outliers(series):
    """
    Substitui valores considerados outliers por NaN em uma série.
    """
    # Calcular o primeiro e terceiro quartis
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    # Calcular o IQR
    IQR = Q3 - Q1
    # Definir limites para considerar um dado como outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Substituir os outliers por NaN
    series[(series < lower_bound) | (series > upper_bound)] = np.nan
    return series

# 1.1. Construindo a função que seleciona as colunas, apaga os outliers e remove nulos
def anula_outliers_colunas_selecionadas(df, colunas, intervalo_de_dias):
    """
    Remove outliers de colunas selecionadas e apaga linhas com valores nulos no DataFrame.
    
    Parâmetros:
    - df (pd.DataFrame): DataFrame de entrada.
    - colunas (list): Lista de colunas a serem processadas.
    - intervalo_de_dias (int): Intervalo de dias para agrupar dados antes de detectar outliers.
    
    Retorno:
    - pd.DataFrame: DataFrame com outliers substituídos por NaN e linhas nulas removidas.
    """
    # Garantir que a coluna 'Data da Coleta' está no DataFrame e em formato datetime
    if 'Data da Coleta' not in df.columns:
        raise ValueError("A coluna 'Data da Coleta' deve estar presente no DataFrame.")
    
    df['Data da Coleta'] = pd.to_datetime(df['Data da Coleta'], format='%Y%m%d')
    
    # Filtrar as colunas desejadas junto com 'Data da Coleta'
    colunas_modificadas = colunas.copy()
    if 'Data da Coleta' not in colunas_modificadas:
        colunas_modificadas.append('Data da Coleta')
    
    temp_df = df[colunas_modificadas].copy()
    temp_df.set_index('Data da Coleta', inplace=True)
    
    # Agrupar dados por intervalo de dias e aplicar a função para substituir outliers por NaN
    temp_df = temp_df.resample(f'{intervalo_de_dias}D').apply(lambda group: group.apply(anula_outliers))
    
    # Remover linhas com valores nulos em qualquer coluna
    temp_df.dropna(inplace=True)
    
    # Resetar o índice para voltar 'Data da Coleta' como uma coluna
    temp_df.reset_index(inplace=True)
    
    # Converter 'Data da Coleta' para o formato aaaammdd como int
    temp_df['Data da Coleta'] = temp_df['Data da Coleta'].dt.strftime('%Y%m%d').astype(int)
    
    # Retornar apenas as colunas originalmente solicitadas
    if 'Data da Coleta' not in colunas:
        temp_df = temp_df[colunas]
    
    return temp_df

In [4]:
df.columns

Index(['Data da Coleta', 'Valor de Venda Gasolina (p/L)',
       'Valor de Venda Etanol (p/L)', 'Valor de Venda Diesel (p/L)',
       'Valor de Venda GLP (p/13kg)', 'Cotação dolar em Real (Compra)',
       'Cotação dolar em Real (Venda)', 'Cotação riyal em Real (Compra)',
       'Cotação riyal em Real (Venda)', 'Fechamento ARAMCO (em Riyais)',
       'Volume de transações ARAMCO (Em milhares)',
       'Fechamento CHEVRON (em Reais)',
       'Volume de transações CHEVRON (Em milhares)',
       'Fechamento EXXON (em Reais)',
       'Volume de transações EXXON (Em milhares)',
       'Fechamento PETR3 (em Reais)',
       'Volume de transações PETR3 (Em milhares)',
       'Fechamento PETR4 (em Reais)',
       'Volume de transações PETR4 (Em milhares)'],
      dtype='object')

In [5]:
scaler = StandardScaler()

In [6]:
# Normalização
features = ['Data da Coleta', 'Fechamento ARAMCO (em Riyais)']

target = ['Valor de Venda Gasolina (p/L)']

temp_df = anula_outliers_colunas_selecionadas(df, features + target,
                                              intervalo_de_corte_outliers)

# Garantir que os dados estejam ordenados por data
temp_df = temp_df.sort_index()

# Separar as features e o target
X = temp_df[features]
y = temp_df[target[0]]

# Separar os dados para treino (exceto os últimos 4 dias) e teste (últimos 4 dias)
train_size = len(temp_df) - 4
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Treinamento do modelo
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões para os últimos 4 dias
y_pred = model.predict(X_test)

# Avaliar o desempenho
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE para os últimos 4 dias: {mae}")

# Exibir previsões e valores reais
predictions = pd.DataFrame({
    'Data': X_test['Data da Coleta'].values,  # Supondo que o índice seja uma data ou identificador
    'Target Real': y_test.values,
    'Target Previsto': y_pred
})
print(predictions)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 488
[LightGBM] [Info] Number of data points in the train set: 741, number of used features: 2
[LightGBM] [Info] Start training from score 5.479384
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [7]:
# Normalização
features = ['Data da Coleta', 'Fechamento ARAMCO (em Riyais)']

target = ['Valor de Venda Etanol (p/L)']

temp_df = anula_outliers_colunas_selecionadas(df, features + target,
                                              intervalo_de_corte_outliers)

# Garantir que os dados estejam ordenados por data
temp_df = temp_df.sort_index()

# Separar as features e o target
X = temp_df[features]
y = temp_df[target[0]]

# Separar os dados para treino (exceto os últimos 4 dias) e teste (últimos 4 dias)
train_size = len(temp_df) - 4
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Treinamento do modelo
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões para os últimos 4 dias
y_pred = model.predict(X_test)

# Avaliar o desempenho
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE para os últimos 4 dias: {mae}")

# Exibir previsões e valores reais
predictions = pd.DataFrame({
    'Data': X_test['Data da Coleta'].values,  # Supondo que o índice seja uma data ou identificador
    'Target Real': y_test.values,
    'Target Previsto': y_pred
})
print(predictions)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 482
[LightGBM] [Info] Number of data points in the train set: 743, number of used features: 2
[LightGBM] [Info] Start training from score 4.175705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [8]:
# Normalização
features = ['Data da Coleta', 'Fechamento ARAMCO (em Riyais)']

target = ['Valor de Venda Diesel (p/L)']

temp_df = anula_outliers_colunas_selecionadas(df, features + target,
                                              intervalo_de_corte_outliers)

# Garantir que os dados estejam ordenados por data
temp_df = temp_df.sort_index()

# Separar as features e o target
X = temp_df[features]
y = temp_df[target[0]]

# Separar os dados para treino (exceto os últimos 4 dias) e teste (últimos 4 dias)
train_size = len(temp_df) - 4
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Treinamento do modelo
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões para os últimos 4 dias
y_pred = model.predict(X_test)

# Avaliar o desempenho
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE para os últimos 4 dias: {mae}")

# Exibir previsões e valores reais
predictions = pd.DataFrame({
    'Data': X_test['Data da Coleta'].values,  # Supondo que o índice seja uma data ou identificador
    'Target Real': y_test.values,
    'Target Previsto': y_pred
})
print(predictions)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 484
[LightGBM] [Info] Number of data points in the train set: 732, number of used features: 2
[LightGBM] [Info] Start training from score 5.131666
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [9]:
# Normalização
features = ['Data da Coleta', 'Fechamento ARAMCO (em Riyais)']

target = ['Valor de Venda GLP (p/13kg)']

temp_df = anula_outliers_colunas_selecionadas(df, features + target,
                                              intervalo_de_corte_outliers)

# Garantir que os dados estejam ordenados por data
temp_df = temp_df.sort_index()

# Separar as features e o target
X = temp_df[features]
y = temp_df[target[0]]

# Separar os dados para treino (exceto os últimos 4 dias) e teste (últimos 4 dias)
train_size = len(temp_df) - 4
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Treinamento do modelo
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões para os últimos 4 dias
y_pred = model.predict(X_test)

# Avaliar o desempenho
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE para os últimos 4 dias: {mae}")

# Exibir previsões e valores reais
predictions = pd.DataFrame({
    'Data': X_test['Data da Coleta'].values,  # Supondo que o índice seja uma data ou identificador
    'Target Real': y_test.values,
    'Target Previsto': y_pred
})
print(predictions)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 477
[LightGBM] [Info] Number of data points in the train set: 734, number of used features: 2
[LightGBM] [Info] Start training from score 95.363103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai